In [ ]:
import os
import json
import requests
from datetime import datetime, timedelta

# === CONFIGURACIÓN ===
INEGI_TOKEN   = "ab6744a6-fdeb-064f-1034-691048c15ab0"
BANXICO_TOKEN = "7c7245244cd2df18b2b03e0834258450b2ab7c578910115fb8975a7f1c48b9e8"

# El indicador que validaste manualmente
SERIE_INEGI_ID = "735879"

CACHE_FILE = "cache_pib_final.json"
CACHE_TTL_HOURS = 12

def cache_load():
    if not os.path.exists(CACHE_FILE): return None
    try:
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            obj = json.load(f)
        ts = datetime.fromisoformat(obj.get("_ts", "1970-01-01"))
        if datetime.now() - ts > timedelta(hours=CACHE_TTL_HOURS): return None
        return obj
    except: return None

def cache_save(data):
    try:
        data["_ts"] = datetime.now().isoformat()
        with open(CACHE_FILE, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
    except: pass

def get_inegi_data_federated(ind_id):
    """
    Usa la ruta BIE-BISE descubierta por el usuario para evitar errores 500.
    """
    # NOTA: Usamos 'true' en metadatos igual que en tu prueba exitosa
    base_url = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml"
    url = f"{base_url}/INDICATOR/{ind_id}/es/00/true/BIE-BISE/2.0/{INEGI_TOKEN}?type=json"

    try:
        r = requests.get(url, timeout=45)
        r.raise_for_status()
        js = r.json()

        # Navegamos la estructura exacta que compartiste
        series = js.get("Series")
        if not series:
            raise ValueError("JSON válido pero sin campo 'Series'")

        obs = series[0].get("OBSERVATIONS")
        if not obs:
            raise ValueError("La serie no tiene observaciones.")

        # Ordenamos para asegurar que tomamos el último (2025/03 en tu caso)
        # La API suele devolver ordenado, pero por seguridad:
        obs_sorted = sorted(obs, key=lambda x: x["TIME_PERIOD"])

        return obs_sorted[-1] # Retornamos el último objeto

    except Exception as e:
        raise RuntimeError(f"Error INEGI: {e}")

def get_dolar_fix():
    """Obtiene el FIX oportuno de Banxico para la conversión"""
    url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos/oportuno"
    try:
        r = requests.get(url, headers={"Bmx-Token": BANXICO_TOKEN}, timeout=15)
        if not r.ok: # Fallback query param
            r = requests.get(f"{url}?token={BANXICO_TOKEN}", timeout=15)
        r.raise_for_status()
        return float(r.json()["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        print("Advertencia: No se pudo obtener FIX. Usando 20.50 estimado.")
        return 20.50

def main(force_refresh=False):
    data = None if force_refresh else cache_load()

    if not data:
        try:
            # 1. Obtener dato INEGI
            ultimo_dato = get_inegi_data_federated(SERIE_INEGI_ID)

            # Parsear valores
            periodo = ultimo_dato["TIME_PERIOD"] # Ej: "2025/03"
            valor_mxn = float(ultimo_dato["OBS_VALUE"]) # Ej: 25411486.26

            # 2. Obtener Dólar
            fix = get_dolar_fix()

            # 3. Conversión
            # El dato viene en Millones de Pesos (UNIT: 1054 en tu JSON)
            # Para pasar a "Miles de Millones de USD" (Billions):
            valor_usd_billions = (valor_mxn / fix) / 1000.0

            data = {
                "periodo": periodo,
                "valor_mxn_millones": valor_mxn,
                "valor_usd_billions": valor_usd_billions,
                "fix": fix,
                "update_inegi": datetime.now().strftime("%Y-%m-%d")
            }
            cache_save(data)

        except Exception as e:
            print(f"Error crítico: {e}")
            return

    # Formateo de salida
    # 2025/03 -> Año 2025, Trimestre 3
    anio, q = data["periodo"].split("/")
    # Mapeo simple: 01=1º, 02=2º, 03=3º, 04=4º
    trimestre_str = f"{int(q)}º"

    print(f"PIB: {data['valor_usd_billions']:,.2f} (USD$ miles de millones)")
    print(f"Periodo: {trimestre_str} Trimestre de {anio}")
    print("-" * 50)
    print(f"Detalles:")
    print(f"Valor de origen (MXN): {data['valor_mxn_millones']:,.2f}")
    print(f"Tipo de cambio usado: ${data['fix']}")
    print("-" * 50)

if __name__ == "__main__":
    # Force refresh para asegurar que probamos tu nueva ruta
    main(force_refresh=True)

PIB: 1,388.04 (USD$ miles de millones) al 3º trimestre de 2025
